# huntabyte/bits-ui

> 

In [ ]:

from lovely_docs.settings import settings, GitSource

from lovely_docs.docs import (
    build_markdown_doc_tree,
    process_tree_depth_first,
    calculate_total_usage,
    calculate_cost,
    save_processed_documents
)
import shutil
from pathlib import Path
import logging
import subprocess
import os

from lovely_docs.git import clone_repo

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

#### Clone the repo

In [ ]:
build = True

source = GitSource(
    name="huntabyte/bitsui",
    doc_dir="docs/static/docs",
    repo="https://github.com/huntabyte/bits-ui",
    commit="main",
    ecosystems=["svelte"])

commit, clone_dir = clone_repo(source)
source.commit = commit # Replace the literal commit (master) with the hash

66: 23976.0/23976.0 12.68 MiB | 6.73 MiB/s

In [ ]:
if build:
    with open("huntabyte-bitsui.log", "w") as log_file:
        # Clean the repo
        subprocess.run(["git", "clean", "-dfx"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in root
        subprocess.run(["pnpm", "install"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Build the package
        subprocess.run(["pnpm", "build"], cwd=clone_dir, stdout=log_file, stderr=log_file, check=True)

        # Install dependencies in docs
        subprocess.run(["pnpm", "install"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

        # Build the docs
        subprocess.run(["pnpm", "build"], cwd=clone_dir / "docs", stdout=log_file, stderr=log_file, check=True)

In [ ]:
# Recursively copy all .txt files in docs/static to corresponding .md files
static_dir = clone_dir / "docs" / "static" / "docs"
for txt_file in static_dir.rglob("*.txt"):
    md_file = txt_file.with_suffix(".md")
    md_file.parent.mkdir(parents=True, exist_ok=True)
    # logger.debug(f"{txt_file} -> {md_file}")
    shutil.copy(txt_file, md_file)

os.remove(clone_dir / "docs" / "static" / "docs"/ "llms.md")

In [ ]:
tree = build_markdown_doc_tree(clone_dir/source.doc_dir)

In [ ]:
# Path("bits.json").write_text(tree.model_dump_json(indent=2))

In [ ]:
# tree.subdirs = tree.subdirs[:3]
# tree.subdirs[1].subdirs = tree.subdirs[1].subdirs[:4]

#### Split the single llms.txt file into pages and directories based on heading levels

In [ ]:
processed_tree = await process_tree_depth_first(settings, tree, source.name)

+269.358s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/accordion/llms.md
+0.311s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/alert-dialog/llms.md
+0.017s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/aspect-ratio/llms.md
+0.015s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/avatar/llms.md
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/button/llms.md
+0.017s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/calendar/llms.md
+0.015s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/checkbox/llms.md
+0.015s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/collapsible/llms.md
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/combobox/llms.md
+0.016s DEBUG lovely_docs/docs.py:157 llm_process_page Processing components/command/llms.md
+0.017s DEBUG lovely_docs/docs.py:157 llm_process_p

#### Calculate cost

In [ ]:
usage =calculate_total_usage(processed_tree)
print(f"\nTotal Usage:")
print(f"  Input tokens: {usage.input:,}")
print(f"  Output tokens: {usage.output:,}")
print(f"  Total tokens: {(usage.input + usage.output):,}")

cost, input_cost, output_cost = calculate_cost(usage, 1, 5)

print(f"Cost:")
print(f"  Total: ${cost:.2f}")
print(f"  Input: ${input_cost:.2f}")
print(f"  Output: ${output_cost:.2f}")


Total Usage:
  Input tokens: 556,576
  Output tokens: 117,018
  Total tokens: 673,594
Cost:
  Total: $1.14
  Input: $0.56
  Output: $0.59


#### save the results

In [ ]:
save_processed_documents(source, settings.output_dir / source.name.replace("/", "_"), processed_tree)